In [ ]:
import json
import gzip
import jsonlines
import yaml
from cdapython import Q, columns, unique_terms, query
Q.set_host_url("http://35.192.60.10:8080")
from collections import defaultdict

In [ ]:
import yaml
from yaml import Loader
from collections import defaultdict
class data_query_validation:
    def __init__(self, mapping_yaml) -> None:
        # map of field name -> set of values
        self.mapping_yaml = mapping_yaml
        self.entity_fields = self._init_entity_fields()
    def _init_entity_fields(self):
        return_dict = {}
        MandT = yaml.load(open(self.mapping_yaml, "r"), Loader=Loader)
        for entity, MorT_dict in MandT.items():
            return_dict[entity.replace('_merge','')] = MorT_dict['Mapping']
        return return_dict
    def two_matching_recs(self, recs, entity_name):
        diffs = {}
        same = True
        rec_sources = list(recs.keys())
        for field in self.entity_fields[entity_name]:
            rec_compare = [recs[source].get(field) for source in rec_sources]
            #type_of_compare = ''
            for i in rec_compare:
                if isinstance(rec_compare[0],list) and isinstance(rec_compare[1],list):
                    if not isinstance(rec_compare[0][0],dict):
                        rec_compare[0].sort()
                        rec_compare[1].sort()
                if rec_compare[0] != rec_compare[1]:
                    if isinstance(rec_compare[1],int) and isinstance(rec_compare[0], str):
                        if int(rec_compare[0])== rec_compare[1]:
                            continue
                    if None not in rec_compare:
                        same = False
                        diffs[field] = {rec_sources[0]:rec_compare[0],
                                        rec_sources[1]:rec_compare[1]}
        return same, diffs
    def all_identical_dups(self, results_dict, entity_name):
        # determines if all duplicate entities are identical
        all_identical = True
        all_diffs = defaultdict(list)
        for idd, recs in results_dict.items():
            for i in range(1, len(recs)):
                same, diffs = self.two_matching_recs({0:recs[0], i:recs[i]}, entity_name)
                if not same:
                    all_identical = False
                    all_diffs[idd].append(diffs)
                else:
                    all_diffs[idd].append(None)
        return all_identical, all_diffs
                    
            
    def two_matching_queries(self, data_file_results, query_results, entity):
        df_ids = set([i['id'] for i in data_file_results])
        qr_ids = set([i['id'] for i in query_results])
        return_same_ids = len(df_ids) == len(qr_ids)
        return_same_num_recs = len(data_file_results)==len(query_results)
        dups_in_data_return = len(df_ids) == len(data_file_results)
        dups_in_query_return = len(qr_ids) == len(query_results)
        no_dups = dups_in_data_return and dups_in_query_return
        missing_ids = []
        added_ids = []
        ids_w_diff_data = []
        #Simple comparison - same ids and number of entities returned
        if return_same_num_recs and return_same_ids and no_dups:
            df_recs = {}
            qr_recs = {}
            for idd in df_ids:
                for rec in data_file_results:
                    if rec['id'] == idd:
                        df_recs[idd] = rec
                        #print('id: ' + idd)
                        #print('age_at_diagnosis: ' + str(rec['age_at_diagnosis']))
                        #print(type(rec['age_at_diagnosis']))
                        break
                for rec in query_results:
                    if rec['id'] == idd:
                        qr_recs[idd] = rec
                        break
                        
                match, diffs = self.two_matching_recs({'query1': df_recs[idd], 
                                                       'query2': qr_recs[idd]}, entity)
                if not match:
                    print(idd)
                    print(diffs)
        #elif len(data_file_results) != len(query_results) and df_ids == qr_ids:
            # There must be duplicates if distinct ids are the same.
            # Reduce dups in longer file - check that data is the same for the dups
            # If not then need to report that error as well.
        else:
            print('fucked')
        return None
    
    def two_data_structs(self, struct1, struct2, total_structure):
        entity_recs
        for entity, tree in total_structure.items():
            
    def count_all_entities(self, q, print_dfs = False):
        q_dict = {'Subjects': q.subject,
                  'ResearchSubjects': q.researchsubject,
                  'Specimens': q.specimen,
                  'Diagnoses': q.diagnosis,
                  'Treatments': q.treatment
                 }
        q_return = {}
        for entity_name, q_call in q_dict.items():
            q_return[entity_name]= q_call.count.run()
        
        if print_dfs:
            for entity_name, q_call in q_return.items():
                print(entity_name + ': ')
                print(q_call)
        return q_return
        
    def all_q_entities(self, q):
        q_dict = {'Subjects': q.subject,
                  'ResearchSubjects': q.researchsubject,
                  'Specimens': q.specimen,
                  'Diagnoses': q.diagnosis,
                  'Treatments': q.treatment
                 }
        q_return = defaultdict(list)
        for entity_name, q_call in q_dict.items():
            for page in q_call.run().paginator():
                q_return[entity_name].extend(page)
            print(entity_name + ": " + str(len(q_return[entity_name])))
        return q_return
    
    def all_q_files(self, q):
        qresults = defaultdict(list)
        for page in q.subject.file.run().paginator():
            qresults['Subject_Files'].extend(page)
        for page in q.researchsubject.file.run().paginator():
            qresults['ResearchSubject_Files'].extend(page)
        for page in q.specimen.file.run().paginator():
            qresults['Specimen_Files'].extend(page)
        return qresults

In [ ]:
dqv = data_query_validation('../GDC_file_endpoint_mapping.yml')

In [ ]:
help(Q.count)

In [ ]:
help(Q.AND)
help(Q.OR)
help(Q.FROM)

In [ ]:
def remove_dups(returned_dict):
    specimens_copy = []
    spec_seen = []
    for specimen in returned_dict['Specimens']:
        if specimen['id'] not in spec_seen:
            specimens_copy.append(specimen)
            spec_seen.append(specimen['id'])
    returned_dict['Specimens'] = specimens_copy
    returned_dict['Subject_Files'] = list(set(returned_dict['Subject_Files']))
    returned_dict['ResearchSubject_Files'] = list(set(returned_dict['ResearchSubject_Files']))
    returned_dict['Specimen_Files'] = list(set(returned_dict['Specimen_Files']))
    return returned_dict
def get_data_counts(returned_dict):
    counts = {}
    for entity_name, entity_recs in returned_dict.items():
        #print(entity_name + ": " + str(len(entity_recs)))
        counts[entity_name] = len(entity_recs)
    return counts
def get_files_entities(returned_dict):
    files_data_file = 'all_Files_3_0.jsonl.gz'
    return_files = defaultdict(list)
    with gzip.open(files_data_file, 'r') as subj:
        reader = jsonlines.Reader(subj)
        for file in reader:
            if file['id'] in returned_dict['Subject_Files']:
                return_files['Subject_Files'].append(file)
                returned_dict['Subject_Files'].remove(file['id'])
                if file['id'] in returned_dict['ResearchSubject_Files']:
                    return_files['ResearchSubject_Files'].append(file)
                    returned_dict['ResearchSubject_Files'].remove(file['id'])
                    if file['id'] in returned_dict['Specimen_Files']:
                        return_files['Specimen_Files'].append(file)
                        returned_dict['Specimen_Files'].remove(file['id'])
            if returned_dict['Subject_Files'] == []:
                break
    returned_dict.update(return_files)
    return returned_dict

In [ ]:

# WHERE
#    (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I'))
#        OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II')))
#      AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
subjects_data_file = '../all_Subjects_3_0.jsonl.gz'
returned_data = defaultdict(list)

pass_all_filters = False
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        for RS in subject.get('ResearchSubject'):
            pass_all_filters = False
            RS_filter = RS.get('primary_diagnosis_site')
            if RS_filter is not None and  RS_filter.upper() == 'Kidney'.upper() and RS.get('Diagnosis') is not None:
                for Diagnosis in RS.get('Diagnosis'):
                    diag_stage = Diagnosis.get('stage')
                    if diag_stage is not None and diag_stage.upper() in ['Stage I'.upper(), 'Stage II'.upper()]:
                        pass_all_filters = True
                        subj_pass_filters = True
                        returned_data['Diagnoses'].append(Diagnosis)
            if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                RS_specimens = RS.pop('Specimen')
                returned_data['Specimens'].extend(RS_specimens)
                for spec in RS_specimens:
                    returned_data['Specimen_Files'].extend(spec.get('Files',[]))
            if pass_all_filters:
                returned_data['ResearchSubjects'].append(RS)
                returned_data['ResearchSubject_Files'].extend(RS['Files'])
        if subj_pass_filters:
            returned_data['Subjects'].append(subject)
            returned_data['Subject_Files'].extend(subject['Files'])


In [ ]:
returned_data = remove_dups(returned_data)
dcounts = get_data_counts(returned_data)
print(dcounts)

In [ ]:
returned_data = get_files_entities(returned_data)

In [ ]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.OR(q2)
q = q_diag.AND(q3)
#qcounts = dqv.count_all_entities(q, print_dfs = True)

In [ ]:
columns().to_list()

In [ ]:
qresults = dqv.all_q_entities(q)

In [ ]:
qresults['Subjects']

In [ ]:
qcounts = get_data_counts(qresults)
print(qcounts)
dcounts = get_data_counts(returned_data)
print(dcounts)

In [ ]:
qresults2 = qresults['Subjects'][0:4]
dqv.two_matching_queries(qresults2, qresults['Subjects'], 'Patient')

In [ ]:
dqv.two_matching_queries(returned_data['ResearchSubjects'], qresults['ResearchSubjects'], 'ResearchSubject')

In [ ]:
dqv.two_matching_queries(returned_data['Diagnoses'], qresults['Diagnoses'], 'Diagnosis')

In [ ]:
dqv.two_matching_queries(returned_data['Specimens'], qresults['Specimens'], 'Specimen')

In [ ]:
dqv.two_matching_queries([], qresults['Treatments'], 'Treatment')

In [ ]:
qresults.update(dqv.all_q_files(q))

In [ ]:
qrs_files=[]
qrs_files_dict = defaultdict(list)
for file in qresults['ResearchSubject_Files']:
    if file['id'] not in qrs_files:
        qrs_files_dict[file['id']].append(file)
        qrs_files.append(file['id'])
print(len(qrs_files))
print(len(qresults['ResearchSubject_Files']))
qrs_files_distinct = []
for file_id, rec_lst in qrs_files_dict.items():
    qrs_files_distinct.append(rec_lst[0])

In [ ]:
dqv.two_matching_queries(returned_data['ResearchSubject_Files'], qrs_files_distinct, 'File')

In [ ]:
qspec_files=[]
qspec_files_dict = defaultdict(list)
for file in qresults['Specimen_Files']:
    if file['id'] not in qspec_files:
        qspec_files_dict[file['id']].append(file)
        qspec_files.append(file['id'])
print(len(qspec_files))
print(len(qresults['Specimen_Files']))
qspec_files_distinct = []
for file_id, rec_lst in qspec_files_dict.items():
    qspec_files_distinct.append(rec_lst[0])

In [ ]:
dqv.two_matching_queries(returned_data['Specimen_Files'], qspec_files_distinct, 'File')

In [ ]:
#Usage test query 1 from ReadTheDocs
#q1 = Q('ResearchSubject.Diagnosis.Treatment.treatment_type = "Radiation Therapy, NOS"')
#q2 = Q('ResearchSubject.identifier.system = "PDC"')
#q3 = Q('ResearchSubject.identifier.system = "GDC"')
#q = q2.From(q1.And(q3))
# Above query requires GDC Diagnosis Treatment type to be Radiation Therapy, NOS, but also have PDC information 
# At least that is how the query gets written. Funny enough PDC does not populate Treatment entities so another
# filter requiring both PDC and GDC to have that info would give zero results.
# Donovan's edit: I'm not sure yet. Keep as is?
subjects_data_file = 'all_Subjects_3_0.jsonl.gz'
returned_data = defaultdict(list)

with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        has_GDC_RS = False
        has_PDC_RS = False
        for RS in subject['ResearchSubject']:
            RS_filter = RS['identifier']['system']
            if RS_filter == 'GDC' and RS.get('Diagnosis') is not None:
                for Diagnosis in RS.get('Diagnosis'):
                    if Diagnosis.get('Treatment') is not None:
                        for Treatment in Diagnosis.get('Treatment'):
                            pass_all_filters = True
                            subj_pass_filters = True
                            break
            if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                RS_specimens = RS.pop('Specimen')
                specimens.extend(RS_specimens)
                for spec in RS_specimens:
                    specimen_files.extend(spec.get('Files',[]))
            if pass_all_filters:
                researchsubjects.append(RS)
                rs_files.extend(RS['Files'])
        if subj_pass_filters:
            #subject.pop('ResearchSubject')
            subjects.append(subject)
            subject_files.extend(subject['Files'])



In [ ]:
# Usage test query 2
#q1 = Q('ResearchSubject.associated_project = "TCGA-BRCA"')
#q2 = Q('days_to_birth > -50*365')
#q3 = Q('identifier.system = "IDC"')
#q = q3.From(q1.And(q2))
# Subject over 50, has an RS with project "TCGA-BRCA" ugh... IDC does not list project as TCGA-BRCA, but tcga_brca
# I really need to be more vigilant with testing and writing these queries
# Donovan's edit: change associated project to tcga_brca
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []

with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        #print(subject.get('days_to_birth', 0.0))
        if subject.get('days_to_birth', 0.0) is not None and int(subject.get('days_to_birth', 0.0)) < -50*365:
            for RS in subject['ResearchSubject']:
                RS_filter = RS.get('member_of_research_project')
                if RS_filter is not None and  RS_filter.upper() == 'tcga_brca'.upper():
                    pass_all_filters = True
                    subj_pass_filters = True
                if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                    RS_specimens = RS.pop('Specimen')
                    specimens.extend(RS_specimens)
                    for spec in RS_specimens:
                        specimen_files.extend(spec.get('Files',[]))
                if pass_all_filters:
                    researchsubjects.append(RS)
                    rs_files.extend(RS['Files'])
        if subj_pass_filters:
            #subject.pop('ResearchSubject')
            subjects.append(subject)
            subject_files.extend(subject['Files'])



In [ ]:
# Remove duplicate Specimens - Cannot have duplicate RS or Subjects
specimens_copy = []
spec_seen = []
ind = []
for specimen in specimens:
    if specimen['id'] not in spec_seen:
        specimens_copy.append(specimen)
subject_files = list(set(subject_files))
rs_files = list(set(rs_files))
specimen_files = list(set(specimen_files))

In [ ]:
print(len(subjects))
print(len(researchsubjects))
print(len(specimens_copy))
print(len(subject_files))
print(len(rs_files))
print(len(specimen_files))

In [ ]:
#Find data from patients with lung cancer under 45 years, 
# males with no smoking exposure with Radiomic (CT, PET images) data, mass spec data, WES or WGS
# Donovan's edit: We do not currently have exposures in our data model. 
#                 We can try to do the rest of the filters though.
q1 = Q('days_to_birth > -50*365')

qsub = q1.subjects()
print(qsub)
print(q1.to_json())

In [ ]:
#FInd RNA seq and proteomics data for patients under age 16 from tissue samples of the brain from CPTAC program.


In [ ]:
#Find data for patients from TCGA-COAD with a Diagnosis of stage iii or greater and morphology 8140/3 across ethnicities.

In [ ]:
subjects_data_file = 'all_Subjects_3_0_w_RS.jsonl.gz'
sample_case_mapping = defaultdict(list)
#WHERE
#    (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I'))
#        OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II')))
#      AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
pass_all_filters = False
all_subjects_specimens = []
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        for RS in subject['ResearchSubject']:
            for spec in RS.get('Specimen',[]):
                if subject['id'] in sub_ids:
                    all_subjects_specimens.append(spec)
                sample_case_mapping[spec['id']].append(RS['id'])


In [ ]:
rs_specs = []
for spec,cases in sample_case_mapping.items():
    if len(cases) > 1:
        print(sample_case_mapping[spec])
    if cases[0] in rs_ids:
        rs_specs.append(spec)

In [ ]:
with gzip.open('IDC_V8_Subjects_Full.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'w') as subj:
        writer = jsonlines.Writer(subj)
        for subject in reader:
            for RS in subject['ResearchSubject']:
                RS['File'] = subject['File']
            writer.write(subject)

In [ ]:
with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        print(subject['ResearchSubject'])
        break

In [ ]:
with gzip.open('gdc.all_cases.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        for spec in subject['samples']:
            print(spec)
            break
        break

In [ ]:
with gzip.open('data/pdc.files-specimens-cases.json.gz', "r") as f_in:
    files_per_sample_dict = json.load(f_in)
specimen_w_out_files = {}
for spec in files_per_sample_dict:
    if files_per_sample_dict[spec] is None:
        specimen_w_out_files[spec] = files_per_sample_dict[spec]
print(len(specimen_w_out_files))

In [ ]:
#files_per_sample_dict['3b96e558-b060-48ec-a1d5-98554c53afc2']
files_per_sample_dict['cecdcea4-7b7e-40fb-a013-e4db87604f67']

In [ ]:
with open('gdc.all_cases.H.jsonl', 'r') as subj:
    for subject in subj:
        line = json.loads(subject)
        print(line)
        break

In [ ]:
type(line)

In [ ]:
lst = ['a', 'b', 'c']
for i,j in enumerate(lst):
    print(i)

In [ ]:
blah = {'a':1,'b':2}
while len(blah)>0:
    list(blah.keys())[0]
    blah.pop(list(blah.keys())[0])
blah

In [ ]:
#blah.pop('b')
blah
len(blah)

In [ ]:
count = 0
seen_subjects = {}
with gzip.open('test_merge_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjects:
            seen_subjects[subject['id']] += 1
        else:
            seen_subjects[subject['id']] = 1
print(count)

In [ ]:
for key, val in seen_subjects.items():
    if val > 1:
        print(key)

In [ ]:
q1 = Q('ResearchSubject.primary_diagnosis_site = "ovary"')
q1.subject.run()

In [ ]:
q1.research_subject.run()

In [ ]:
q1.specimen.run(limit=1000)

In [ ]:
specs = q1.specimen.run(limit=1000)

In [ ]:
spec_ids = []
for page in specs.paginator():
    for spec in page:
        if spec['id'] not in spec_ids:
            spec_ids.append(spec['id'])
        else:
            print(spec)

In [ ]:
print(len(spec_ids))
print(len(set(spec_ids)))
print(len(spec_seen))

In [ ]:
subjects_data_file = 'all_Subjects_3_0_w_RS.jsonl.gz'
returned_data = []
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []
diagnoses = []
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        pass_all_filters = False
        subj_pass_filters = False
        for RS in subject['ResearchSubject']:
            pass_all_filters = False
            RS_filter = RS.get('primary_diagnosis_site')
            if RS_filter is not None and  RS_filter.upper() == 'ovary'.upper():
                pass_all_filters = True
                subj_pass_filters = True
                researchsubjects.append(RS)
                specimens.extend(RS.get('Specimen',[]))
                diagnoses.extend(RS.get('Diagnosis',[]))
        if subj_pass_filters:
            subject.pop('ResearchSubject')
            subjects.append(subject)

In [ ]:
specimens_copy = []
spec_seen = []
for specimen in specimens:
    if specimen['id'] not in spec_seen:
        specimens_copy.append(specimen)
        spec_seen.append(specimen['id'])
subject_files = list(set(subject_files))
rs_files = list(set(rs_files))
specimen_files = list(set(specimen_files))

In [ ]:
print(len(specimens))

In [ ]:
print(len(subjects))
print(len(researchsubjects))
print(len(diagnoses))
print(len(specimens_copy))
print(len(subject_files))
print(len(rs_files))
print(len(specimen_files))
sub_ids = []
for subject in subjects:
    sub_ids.append(subject['id'])
rs_ids = []
for rs in researchsubjects:
    rs_ids.append(rs['id'])

In [ ]:
diff = spec_ids==spec_seen
diff

In [ ]:
spec_seen[0]

In [ ]:
res = [x for x in spec_seen + spec_ids if x not in spec_seen or x not in spec_ids]
res

In [ ]:
if None in spec_ids:
    print('why?!')

In [ ]:
spec_ids = []
for page in specs.paginator():
    for spec in page:
        if spec['id'] is None:
            print(spec)

In [ ]:
q1 = Q('ResearchSubject.primary_diagnosis_site = "ovary"')
q2 = Q('ResearchSubject.identifier.system = "IDC"')
q = q1.And(q2)
q.research_subject.run()

In [ ]:
q.specimen.run()

In [ ]:
q_uh_oh = q.specimen.run()
q_uh_oh[0]

In [ ]:
qidcrs = q.research_subject.run()
qidcrs[0]

In [ ]:
count = 0
seen_subjects = {}
with gzip.open('IDC_V8_Subjects_Full_v2.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjects:
            seen_subjects[subject['id']] += 1
        else:
            seen_subjects[subject['id']] = 1
print(count)
print(len(seen_subjects))

In [ ]:
count = 0
seen_subjectsgdc = {}
with gzip.open('gdc.all_cases.A.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsgdc:
            seen_subjectsgdc[subject['id']] += 1
        else:
            seen_subjectsgdc[subject['id']] = 1
print(count)
print(len(seen_subjectsgdc))

In [ ]:
count = 0
seen_subjectspdc = {}
with gzip.open('pdc.all_cases.A.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectspdc:
            seen_subjectspdc[subject['id']] += 1
        else:
            seen_subjectspdc[subject['id']] = 1
print(count)
print(len(seen_subjectspdc))

In [ ]:
all_sub = []
all_sub.extend(seen_subjects)
all_sub.extend(seen_subjectsgdc)
all_sub.extend(seen_subjectspdc)
all_sub = set(all_sub)
len(all_sub)

In [ ]:
for idd, count in seen_subjects.items():
    if count>1:
        print(idd)
        break

In [ ]:
count = 0
seen_subjectsidc = {}
with gzip.open('IDC_V8_Subjects_Full.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsidc:
            seen_subjectsidc[subject['id']] += 1
        else:
            seen_subjectsidc[subject['id']] = 1
print(count)
print(len(seen_subjectsidc))

In [ ]:
count = 0
seen_subjectsidc_ = {}
with gzip.open('idc_v8_Subjects_w_RS.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        count+=1
        if subject['id'] in seen_subjectsidc_:
            seen_subjectsidc_[subject['id']] += 1
        else:
            seen_subjectsidc_[subject['id']] = 1
print(count)
print(len(seen_subjectsidc_))

In [ ]:
for i in seen_subjectsidc_:
    print(i)
    break

In [ ]:
count = 0
seen_subjectsidc = {}
with gzip.open('idc_v8_Subjects.jsonl.gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    with gzip.open('idc_v8_Subjects_v2.jsonl.gz', 'w') as subj_out:
        writer = jsonlines.Writer(subj_out)
        for subject in reader:
            count+=1
            if subject['id'] in seen_subjectsidc:
                seen_subjectsidc[subject['id']] += 1
            else:
                seen_subjectsidc[subject['id']] = 1
            for RS in subject['ResearchSubject']:
                RS['File'] = subject['File']
            writer.write(subject)
print(count)
print(len(seen_subjectsidc))

In [ ]:
count = 0
seen_subjectsidc = {}
max_bit_size = 104857600
import sys
with gzip.open('../../Downloads/public_R3_gcp-merge2.jsonl (2).gz', 'r') as subj:
    reader = jsonlines.Reader(subj)
    break_all = False
    for subject in reader:
        line_size = sys.getsizeof(subject) + sys.getsizeof(subject['File'])
        count += 1
        for rsubject in subject['ResearchSubject']:
            line_size += sys.getsizeof(rsubject) + sys.getsizeof(rsubject['File'])
            if "Specimen" in rsubject:
                for spec in rsubject['Specimen']:
                    line_size += sys.getsizeof(spec) + sys.getsizeof(spec['File'])
            if "Diagnosis" in rsubject:
                for d in rsubject['Diagnosis']:
                    line_size+=sys.getsizeof(d)
        if count in [95823, 95824, 95825]:
            print(subject['identifier'])
            print('num files: ' + str(len(subject['File'])))
            print(sys.getsizeof(subject))
            print(sys.getsizeof(subject['File']))
            print(line_size)
        if count > 95825:
            break

In [ ]:
def create_udf_str(func_desc):
    var_list = ['x','y','z','a','b','c']
    out = "CREATE TEMP FUNCTION "+ func_desc[0] + "("
    for var in range(len(func_desc[1])):
        out += (var_list[var] + " " + func_desc[1][var])
        if var == len(func_desc[1])-1:
            out + ") RETURNS "
        else:
            out += ", "
    out = out + func_desc[2][0] + " AS ("
    if len(func_desc[1]) == 1:
        out = out + func_desc[0]()
    else:
        out = out + func_desc[0](func_desc[1][1:])
    out = out + "); "
    return out
func = ['name', ['string','string', 'string'], 'string']
create_udf_str(func)

In [ ]:
data_file = 'all_Files_3_0.jsonl.gz'
with gzip.open(data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    gdc = open('gdc_drs_uri.txt', 'w')
    pdc = open('pdc_drs_uri.txt', 'w')
    idc = open('idc_drs_uri.txt', 'w')
    for subject in reader:
        if subject['identifier'][0]['system'] == 'GDC':
            gdc.write(subject['drs_uri'])
            gdc.write( '\n')
        elif subject['identifier'][0]['system'] == 'PDC':
            pdc.write(subject['drs_uri'])
            pdc.write( '\n')
        elif subject['identifier'][0]['system'] == 'IDC':
            idc.write(subject['drs_uri'])
            idc.write( '\n')

In [ ]:
red = ['a','b','c']
with open('drs_uri.txt', 'w') as out:
    for subject in red:
        out.write(subject)
        out.write( '\n')

In [ ]:
subjects_data_file = 'gdc.all_cases.jsonl.gz'
break_all = False
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        if subject['case_id'] == 'fef9c64f-5959-4da0-aaa2-66b56fc7b4c3':
            for samp in subject['samples']:
                for portion in samp['portions']:
                    for analyte in portion['analytes']:
                        if analyte['analyte_id'] =='e7224386-1bc6-4e96-afe5-7cac425ed3a9':
                            print(portion['portion_id'])
                            print(analyte)

In [ ]:
from cdatransform.transform.validate import LogValidation
import logging

In [ ]:
logger = logging.getLogger('data_explorer')
logging.basicConfig(
        filename='data_explorer.txt',
        format="%(asctime)s %(levelname)-8s %(message)s",
        level=logging.INFO,
    )

validate = LogValidation()
validate.agree_sources({'sub1': subjects[0], 'sub1.1': subjects[50]}, 
                       subjects[0]['id'],
                       ['species', 'sex', 'race', 'ethnicity', 'days_to_birth',
                        'vital_status',
                        'age_at_death',
                        'cause_of_death',])
validate.generate_report(logger)

In [ ]:
subjects[50]

In [ ]:
blah = data_query_validation('GDC_subject_endpoint_mapping.yml')

In [ ]:
results = blah.all_q_entities(q)

In [ ]:
results.keys()

In [ ]:
counts = blah.count_all_entities(q, print_dfs = True)

In [ ]:
same, diffs = blah.two_matching_recs({'data':subjects[0], 'query': subjects[1]}, 'Patient')
same
diffs

In [ ]:
blah.two_matching_queries(subjects, qsub, 'Patient')

In [ ]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.Or(q2)
q = q_diag.And(q3)

print("Subject")
qsub = q.subject.run()

In [ ]:
qrs = q.researchsubject.run()

In [ ]:
blah.two_matching_queries(researchsubjects, qrs, 'ResearchSubject')

In [ ]:
qspec = q.specimen.run()

In [ ]:
qspec = []
for page in q.specimen.run().paginator():
    qspec.extend(page)
blah.two_matching_queries(specimens, qspec, 'Specimen')
print(len(specimens))
print(len(qspec))


In [ ]:
qdiag = q.diagnosis.run()

In [ ]:
blah.two_matching_queries(diagnoses, qdiag, 'Diagnosis')

In [ ]:
s1 = {'a': [{'id': 'a', 'sex': 'male', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'male', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','s']}, 
            {'id': 'a', 'sex': 'Male', 'race': 'b', 'subject_associated_project': ['q','r']}],
     'b': [{'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']},
            {'id': 'a', 'sex': 'female', 'race': 'w', 'subject_associated_project': ['q','r']}, 
            {'id': 'a', 'sex': 'Male', 'race': 'b', 'subject_associated_project': ['q','r']}]}
blah.all_identical_dups(s1, 'Patient')


In [ ]:
s1 = {'a': [{'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'},
            {'id': 'a', 'sex': 'male', 'race': 'w'}]}
blah.all_identical_dups(s1, 'Patient')

In [ ]:
q1 = Q('ResearchSubject.Diagnosis.stage = "Stage I"')
q2 = Q('ResearchSubject.Diagnosis.stage = "Stage II"')
q3 = Q("ResearchSubject.primary_diagnosis_site = 'Kidney'")
q_diag = q1.Or(q2)
q = q_diag.And(q3)

print("Subject")
qsub = q3.file.run()

In [ ]:
qsub[0]

In [ ]:
query_dict = defaultdict(list)
for i in qsub:
    query_dict[i['id']].append(i)

In [ ]:
len(query_dict)==len(qsub)
print(len(query_dict))
print(len(qsub))

In [ ]:
blahf = data_query_validation('GDC_file_endpoint_mapping.yml')
blahf.all_identical_dups(query_dict, 'File')

In [ ]:
query_dict['ce1b271c-1a5a-450e-97a6-b223f5a6b4a8']

In [ ]:
# WHERE
#    (((UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage I'))
#        OR (UPPER(_ResearchSubject_Diagnosis.stage) = UPPER('Stage II')))
#      AND (UPPER(_ResearchSubject.primary_diagnosis_site) = UPPER('Kidney'))
subjects_data_file = 'all_Subjects_3_0.jsonl.gz'
returned_data = []
subjects = []
subject_files = []
researchsubjects = []
rs_files = []
specimens = []
specimen_files = []
diagnoses = []
filters_dict = {'Subject': "Subject['id'] == 2",
               'ResearchSubject': "ResearchSubject.get('primary_diagnosis')"}

def query_data_file(data_file, filters_dict)
    returned_data = {'Subjects': [],
                    'ResearchSubjects': [],
                    'Diagnoses': [],
                    'Treatments': [],
                    'Specimens': [],
                    'Subject_Files': [],
                    'ResearchSubject_Files': [],
                    'Specimen_Files': []}
    
    with gzip.open(data_file, 'r') as subj:
        reader = jsonlines.Reader(subj)
        for Subject in reader:
            all_lower_filters = {'Subject': {'lower_filters': 
                                             {'ResearchSubject': False,
                                            'Diagnosis': False,
                                            'Treatment': False,
                                            'Specimen': False},
                                             'pass_lower': False,
                                             'pass_entity': False,
                                             
                                            },
                                 'ResearchSubject': {'lower_filters': 
                                             {'Diagnosis': False,
                                            'Treatment': False,
                                            'Specimen': False},
                                             'pass_lower': False,
                                             'pass_entity': False
                                                     
                                            },
                                 'Diagnosis': {'lower_filters': 
                                             {'Treatment': False},
                                             'pass_lower': False,
                                             'pass_entity': False,  
                                            },
                                 'Treatment': {'pass_entity': False},
                                 'Specimen': {'pass_entity': False}
                                 
                                }
            if eval(filters_dict['Subject']):
                subj_pass_filters = True
                for ResearchSubject in Subject.get('ResearchSubject'):
                    if eval(filters_dict['ResearchSubject'])
                    pass_all_filters = False

    pass_all_filters = False
    with gzip.open(subjects_data_file, 'r') as subj:
        reader = jsonlines.Reader(subj)
        for subject in reader:
            pass_all_filters = False
            subj_pass_filters = False
            for RS in subject.get('ResearchSubject'):
                pass_all_filters = False
                RS_filter = RS.get('primary_diagnosis_site')
                if RS_filter is not None and  RS_filter.upper() == 'Kidney'.upper() and RS.get('Diagnosis') is not None:
                    for Diagnosis in RS.get('Diagnosis'):
                        if Diagnosis.get('id') == '5197ce2d-2c37-11ec-b712-0a4e2186f121':
                            print(Diagnosis)
                        diag_stage = Diagnosis.get('stage')
                        if diag_stage is not None and diag_stage.upper() in ['Stage I'.upper(), 'Stage II'.upper()]:
                            pass_all_filters = True
                            subj_pass_filters = True
                            #print(Diagnosis)
                            diagnoses.append(Diagnosis)
                            #
                if pass_all_filters and RS.get('Specimen',[]) is not None and RS.get('Specimen',[]) !=[]:
                    RS_specimens = RS.pop('Specimen')
                    specimens.extend(RS_specimens)
                    for spec in RS_specimens:
                        specimen_files.extend(spec.get('Files',[]))
                if pass_all_filters:
                    researchsubjects.append(RS)
                    rs_files.extend(RS['Files'])
            if subj_pass_filters:
                #subject.pop('ResearchSubject')
                subjects.append(subject)
                subject_files.extend(subject['Files'])

In [ ]:
filters_dict = {'Subject': """Subject['id'] == 2""",
               'ResearchSubject': "ResearchSubject.get('primary_diagnosis')"}
samples = [{'id': 2, 'other': 'blah'}, {'id': 3, 'other': 'blah'}]
ret = []
for Subject in samples:
    print(exec(filters_dict['Subject']))
    if exec(filters_dict['Subject']):
        ret.append(Subject)
ret

In [ ]:
Subject = {'id': 2, 'other': 'blah'}
print(type(filters_dict['Subject']))
print(eval(filters_dict['Subject']))

In [ ]:
import pandas as pd

In [ ]:
data_file = 'all_Subjects_3_0.jsonl.gz'
with gzip.open(data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for Subject in reader:
        df = pd.json_normalize(Subject)
        RS = pd.DataFrame()
        for i in df['ResearchSubject'][0]:
            RS = RS.append(pd.json_normalize(i))
        break
print(type(df))
RS

In [ ]:
def _init_ccdh_field_mapping(cda_mapping_yaml, ccdh_mapping):
    return_dict = {}
    MandT = yaml.load(open(cda_mapping_yaml, "r"), Loader=Loader)
    for entity, MorT_dict in MandT.items():
        #return_dict[entity.replace('_merge','')] = MorT_dict['Mapping']
        return_dict[entity] = {}
        for field in MorT_dict['Mapping']:
            if field in ccdh_mapping.get(entity,{}):
                return_dict[entity][field] = ccdh_mapping[entity][field]
            else:
                return_dict[entity][field] = field
    
    return return_dict

In [ ]:
_init_ccdh_field_mapping('../GDC_file_endpoint_mapping.yml', {'File': {'data_category': 'file_category'}})

In [ ]:
with open('../cdatransform/load/ccdh_map.json', 'r') as f:
    blah = json.load(f)

In [ ]:
blah

In [ ]:
Q('ResearchSubject.primary_diagnosis_site IN ("uterus", "uterus, NOS")').researchsubject.count.run(show_sql=True)

In [ ]:
import gzip
with gzip.open('../gdc.all_files.jsonl.gz','r') as cases:
    reader = jsonlines.Reader(cases)
    for file in reader:
        print(file)
        break
        for rs in subject['Subject']:
            for sample in rs.get('Specimen',[]):
                if isinstance(sample['age_at_collection'],list):
                    print('wtf')
                    print(sample)

        

In [ ]:
import yaml

def csv_cleaner(val):
    if isinstance(val,list):
        return val[0]
    elif val == 'NULL' or val is None:
        return 'NOT CURRENTLY MAPPED'
    elif val == 'files.NOT_APPLICABLE':
        return 'NOT APPLICABLE'
    else:
        return val
files_endpoints = {'GDC': '../GDC_file_endpoint_mapping.yml',
                  'PDC': '../PDC_file_endpoint_mapping.yml',
                  'IDC': '../IDC_mapping.yml'}
subjects_endpoints = {'GDC': '../GDC_subject_endpoint_mapping.yml',
                  'PDC': '../PDC_subject_endpoint_mapping.yml',
                  'IDC': '../IDC_mapping.yml'}

for DC in files_endpoints:
    files_endpoints[DC] = yaml.load(open(files_endpoints[DC], "r"), Loader=Loader)
for DC in subjects_endpoints:
    subjects_endpoints[DC] = yaml.load(open(subjects_endpoints[DC], "r"), Loader=Loader)
subjects_endpoints

header = ['field']
header.extend([dc for dc in list(files_endpoints.keys())])

header
files_csv = []
ent_mapping = {}
for entity in files_endpoints['GDC']:
    ent_mapping[entity] = []
    for field in files_endpoints['GDC'][entity]['Mapping']:
        if field == 'identifier':
            for val in files_endpoints['GDC'][entity]['Mapping'][field]:
                row = {'field': '.'.join([field,val])}
                for DC,mapping in files_endpoints.items():
                    print(DC)
                    print(mapping.get(entity,{}).get('Mapping',{}).get(field))
                    row[DC] = csv_cleaner(mapping.get(entity,{}).get('Mapping',{}).get(field,{}).get(val))
                ent_mapping[entity].append(row)
        else:
            row = {'field': field}
            for DC,mapping in files_endpoints.items():
                row[DC] = csv_cleaner(mapping.get(entity,{}).get('Mapping',{}).get(field))
            ent_mapping[entity].append(row)
ent_mapping

In [ ]:
import csv
for entity in ent_mapping:
    with open('files_csv/'+entity+'.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = header)
        writer.writeheader()
        writer.writerows(ent_mapping[entity])

In [ ]:
data = "../tests/integration/gdc.jsonl.gz"
with gzip.open(data, 'r') as expected_data:
    reader = jsonlines.Reader(expected_data)
    for case in reader:
        if case['case_id'] == "375436b3-66ac-4d5e-b495-18a96d812a69":
            print(case['submitter_id'])
            for sample in case['samples']:
                print(len(sample['files']))

In [ ]:
transformed = {'id': 'TARGET-10-PATCNI', 'identifier': [{'value': 'TARGET-10-PATCNI', 'system': 'GDC'}], 'species': 'homo sapiens', 'sex': None, 'race': None, 'ethnicity': None, 'days_to_birth': None, 'subject_associated_project': ['TARGET-ALL-P3'], 'vital_status': None, 'days_to_death': None, 'cause_of_death': None, 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a'], 'ResearchSubject': [{'id': '7eeced68-1717-4116-bcee-328ac70a9682', 'identifier': [{'value': '7eeced68-1717-4116-bcee-328ac70a9682', 'system': 'GDC'}], 'member_of_research_project': 'TARGET-ALL-P3', 'primary_diagnosis_condition': 'Lymphoid Leukemias', 'primary_diagnosis_site': 'Hematopoietic and reticuloendothelial systems', 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a'], 'Diagnosis': [], 'Specimen': [{'id': '37789082-b4bb-4f18-9a3d-aca1a65c2ced', 'identifier': [{'value': '37789082-b4bb-4f18-9a3d-aca1a65c2ced', 'system': 'GDC'}], 'associated_project': 'TARGET-ALL-P3', 'days_to_collection': None, 'primary_disease_type': 'Lymphoid Leukemias', 'anatomical_site': None, 'source_material_type': 'Primary Blood Derived Cancer - Bone Marrow', 'specimen_type': 'sample', 'derived_from_specimen': 'initial specimen', 'derived_from_subject': 'TARGET-10-PATCNI', 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a']}, {'id': '6388f3ec-eccc-5df4-8558-50bf416b16fa', 'identifier': [{'value': '6388f3ec-eccc-5df4-8558-50bf416b16fa', 'system': 'GDC'}], 'associated_project': 'TARGET-ALL-P3', 'days_to_collection': None, 'primary_disease_type': 'Lymphoid Leukemias', 'anatomical_site': None, 'source_material_type': 'Primary Blood Derived Cancer - Bone Marrow', 'specimen_type': 'portion', 'derived_from_specimen': '37789082-b4bb-4f18-9a3d-aca1a65c2ced', 'derived_from_subject': 'TARGET-10-PATCNI', 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a']}, {'id': '7de4ade2-dc5c-53f6-9815-8e42f2182b59', 'identifier': [{'value': '7de4ade2-dc5c-53f6-9815-8e42f2182b59', 'system': 'GDC'}], 'associated_project': 'TARGET-ALL-P3', 'days_to_collection': None, 'primary_disease_type': 'Lymphoid Leukemias', 'anatomical_site': None, 'source_material_type': 'Primary Blood Derived Cancer - Bone Marrow', 'specimen_type': 'analyte', 'derived_from_specimen': '6388f3ec-eccc-5df4-8558-50bf416b16fa', 'derived_from_subject': 'TARGET-10-PATCNI', 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a']}, {'id': '12cfe700-d1b4-4b88-b509-b8dfaf12ecdf', 'identifier': [{'value': '12cfe700-d1b4-4b88-b509-b8dfaf12ecdf', 'system': 'GDC'}], 'associated_project': 'TARGET-ALL-P3', 'days_to_collection': None, 'primary_disease_type': 'Lymphoid Leukemias', 'anatomical_site': None, 'source_material_type': 'Primary Blood Derived Cancer - Bone Marrow', 'specimen_type': 'aliquot', 'derived_from_specimen': '7de4ade2-dc5c-53f6-9815-8e42f2182b59', 'derived_from_subject': 'TARGET-10-PATCNI', 'Files': ['7bb2674c-3d3d-4e86-90fd-22c1164ffddf', '7ca0dc32-c91a-433c-bffa-fa7004a333a7', '4ba201bd-6832-4b15-bf00-f7647bed5f3a']}]}]}

In [ ]:
dqv.two_matching_queries(test['ResearchSubject'][0]['Specimen'],transformed['ResearchSubject'][0]['Specimen'], 'Specimen')

In [ ]:
dqv.entity_fields['Specimen']['Files'] = 'blah blah blah'

In [ ]:
dqv.entity_fields['Specimen']

In [ ]:
gdc_files = []
with gzip.open('../gdc.all_files.jsonl.gz','r') as files:
    reader = jsonlines.Reader(files)
    for file in reader:
        gdc_files.append(file['file_id'])
len(gdc_files)

In [ ]:
gdc_files = []
with gzip.open('../gdc.all_files.H.jsonl.gz','r') as files:
    reader = jsonlines.Reader(files)
    for file in reader:
        gdc_files.append(file['id'])
len(gdc_files)

In [ ]:
gdc_files = []
with gzip.open('../gdc.all_files.A.jsonl.gz','r') as files:
    reader = jsonlines.Reader(files)
    for file in reader:
        gdc_files.append(file['id'])
len(gdc_files)

In [ ]:
pdc_files = []
with gzip.open('../pdc.all_files.jsonl.gz','r') as files:
    reader = jsonlines.Reader(files)
    for file in reader:
        pdc_files.append(file['file_id'])
len(pdc_files)

In [ ]:
pdc_files = []
with gzip.open('../pdc.all_files.A.jsonl.gz','r') as files:
    reader = jsonlines.Reader(files)
    for file in reader:
        pdc_files.append(file['id'])
len(pdc_files)

In [ ]:
st1 = {'id': 'sub1', 'sex':'male', 'Files':['a','b'], 
       'ResearchSubject':[{'id':'rs1', 
                           'Specimen':[{'id':'spec1'}]
                          }]
      }
st2 = {'id': 'sub1', 'sex':'male', 'Files':['a','b'], 
       'ResearchSubject':[{'id':'rs1', 
                           'Specimen':[{'id':'spec1'}]
                          }]
      }
total_structure = {'Patient': {'id':'str', 'sex': 'str', 'Files': 'str',
                               'ResearchSubject'}}

In [ ]:
def two_data_structs(struct1, struct2, total_structure):
    entity_recs
    for entity, tree in total_structure.items():

In [ ]:
subjects_data_file = '../all_Subjects_3_0.jsonl.gz'
returned_data = defaultdict(list)
#ResearchSubject.primary_diagnosis_site = "brain"
pass_all_filters = False
with gzip.open(subjects_data_file, 'r') as subj:
    reader = jsonlines.Reader(subj)
    for subject in reader:
        subj_pass_filters = False
        for RS in subject.get('ResearchSubject'):
            pass_all_filters = False
            RS_filter = RS.get('primary_diagnosis_site')
            if RS_filter is not None and  RS_filter.upper() == 'brain'.upper():
                pass_all_filters = True
                subj_pass_filters = True
                returned_data['Diagnoses'].extend(RS.get('Diagnosis',[]))
                if RS.get('Specimen',[])!=[]:
                    RS_specimens = RS.pop('Specimen')
                    returned_data['Specimens'].extend(RS_specimens)
                    for spec in RS_specimens:
                        returned_data['Specimen_Files'].extend(spec.get('Files',[]))
                returned_data['ResearchSubjects'].append(RS)
                returned_data['ResearchSubject_Files'].extend(RS['Files'])
        if subj_pass_filters:
            returned_data['Subjects'].append(subject)
            returned_data['Subject_Files'].extend(subject['Files'])



In [ ]:
returned_data = remove_dups(returned_data)
dcounts = get_data_counts(returned_data)
print(dcounts)

In [ ]:
q = Q('ResearchSubject.primary_diagnosis_site = "brain"')
qresults = dqv.all_q_entities(q)
qcounts = get_data_counts(qresults)
print(qcounts)

In [ ]:
with gzip.open('../idc_v9_Subjects_final.jsonl.gz') as idc:
    reader = jsonlines.Reader(idc)
    for i in reader:
        print(i['species'])
        break

In [ ]:
with gzip.open('../all_Subjects_3_0.jsonl.gz') as idc:
    reader = jsonlines.Reader(idc)
    for i in reader:
        if i['species'] == 'Homo sapiens':
            print(i['identifier'])
            break

In [ ]:
fore_ex_specs = [{
  "specimens": {
    "id": "f6cce507-641b-11e8-bcf1-0a2705229b82",
    "identifier": [{
      "system": "PDC",
      "value": "f6cce507-641b-11e8-bcf1-0a2705229b82"
    }],
    "associated_project": "CPTAC-2",
    "days_to_collection": None,
    "primary_disease_type": "Colon Adenocarcinoma",
    "anatomical_site": "Not Reported",
    "source_material_type": "Primary Tumor",
    "specimen_type": "sample",
    "derived_from_specimen": "initial specimen",
    "derived_from_subject": "09CO022",
    "Files": ["0b236baf-d1cc-4d1b-89ab-f47a1baf2f54", "932ca639-c23b-4a5c-be78-5a257f9212e7", "8119ed56-ce52-43a3-a323-c4bf58dd935c", "984427aa-2013-4822-9a04-2e7e3a95e542", "a0f64b2a-3672-49a7-9a45-509d4f89a5d6", "87a94cca-986a-456c-94c4-05d297bd7274", "b576d133-59f8-40a3-8e02-8b6007547d34", "6e8310f5-0bc6-49dd-b934-06722427b421", "06b6f850-b3b6-492b-88a5-214906b71338", "8dea43ca-5b46-4d97-8fb1-ea173bc5c5d3", "85293896-3816-4afa-acb7-bc4311f64adc", "b6b5ce5c-ac41-4be9-9c02-5738509ac2f1", "3ac4db66-7660-451f-833d-32ffa4f159d6", "efcb2319-d4c1-486e-b954-5965ed777dd5", "34f3b35d-056e-458e-805d-aa9eddc8fcc9", "d123999c-98fa-4171-b248-7b782643ab28", "1f63a16d-ee0b-496f-a3cc-52eda4f199b1", "1d03430b-fbb9-4aa4-aa63-0f1d57254bd8", "f07375eb-d4a0-4df9-809c-77cb576ef4b5", "21b0644c-4bc2-4a72-826a-eb0b507add08", "e68b9b17-100e-48c1-84d7-d1a5f9485c1e", "298465a9-a0e0-4691-b368-6a2c00be294f", "c309d2e0-0551-465a-a4e0-ca5e7a59d7f1", "f0d9fd49-298f-449b-9d29-b7253c5e2a7c", "58797074-31b1-4ad2-8c4f-5bcc39486c85", "724c463d-afb5-4ab9-bf70-90505347a10f", "aeee4181-b277-472a-881d-b61bf178f597", "627d302f-557c-4e03-9050-89ee2404d04a", "72af85d8-ca3e-461d-8813-83e4981a7447", "e098e982-8a8f-449b-b36b-de20b63a8dc5", "7aece4d7-687f-428b-998b-92030e2ea0b5", "96ce4701-3f16-47dd-9778-1d8c1eef16e6", "e22e4280-7ac3-4038-888a-095b4ce4a7a6", "0d846fdf-169e-4081-910d-a792fab3df43", "2b85a5e2-7091-4fb3-859e-4520e38e4525", "40617525-424e-4bc0-8468-4e00ff9f0ccb", "5de6ec5d-e824-405d-ab47-bb633f0b30b0", "f89c439d-465d-4b2e-b117-19aa82cd87b4", "29397011-e525-4027-9c3d-c1f86b15e041", "6013ddc3-de68-4fa0-81bb-6c4b30ba06f2", "b63506ea-c6c2-41b4-9c4e-0591f3c5b620", "75b3f79d-a90e-4204-a67d-dd8af9fb2c12", "80de8307-400c-4e91-86c5-110183826a8f", "55dea7f7-c521-4b71-8eb6-4b18fde8dd3e", "d252a407-fb5b-40c4-83e4-aa4865adb9e2", "aff41011-83d2-412d-9ece-887272bbfcb1", "9992288d-7bdc-4cd2-99a8-5eedefbeffca", "22634cb8-c4a5-40a5-a9d6-e447896e8681", "f3bb04cb-8cbd-45b6-8b2c-c3c42a56e9c2", "82a24092-4d20-48be-9242-434b5cb13300", "a6cbebbe-ec64-4fc4-9df9-64b7c83af3e0", "ec2b6628-0081-4609-9d84-8fe7e8fe5ee2", "b93abf59-3591-4921-9695-3cae8898818b", "9e836fe1-d5a3-4c59-ad7a-1c8a084b6a9b", "f214f823-0d83-4563-9cfd-a4e5d421178a", "81c2839d-9f30-4f26-bca7-bc5616b1e7c7", "51906b58-0772-436f-bcb8-fc2f36fcd5e2", "7d3c33cf-5028-4ba0-98f0-c9d29411daf4", "7604aa47-6b24-4615-958b-397ecdc1e5ee", "a53aa3f0-1659-4c46-9c1c-73879ea12985", "47832160-26f2-4ad2-ae65-a781303b21d7", "fbbfc3ab-0238-48e9-8dff-c669e88c54d4", "fa9b4f1c-0297-4b48-8130-250548130a10", "1a1b004e-2ac6-42a7-b745-3e52be7fa22c", "99d06cc3-01d5-4ac1-bd89-cceee73864b5", "6f7cee54-ce35-4e89-9555-c5fe3faf0dd1", "07dde26b-a6b4-4758-8f7d-483bb4340a95", "0daea74c-6886-415b-9395-cd4e0857acb4", "41057289-39ea-4f09-b24f-dfd1336fc1ed", "8e5af17a-02fc-4ba7-9887-bc03b0c83fca", "cfb64703-2cdb-4b35-984f-4d0a582aff06", "2cec8163-ac36-45ba-bdf6-ece8372dd44a", "ebd17416-a9ee-4ce9-9252-76611337a18e", "3f3383b2-97a7-4db3-9055-ef1190adca92", "0a63771a-14b2-4d6b-9c08-c7177c372789", "df3f15b4-15ce-4957-8c30-912eedf71e18", "6ca7d2db-8d2e-4b3a-b296-18b401a8a447", "6e2d5918-e230-433d-9178-7c8fb348d19b", "7f475e53-7efa-45f4-8e34-3bc84cc068b8", "7b831b12-9e11-489f-bd37-47d37d740d4d", "28cebeba-0af6-42f8-9d3d-9b2d6aadcd8a", "cea9dd09-9672-4bf5-b093-0227db5b13f0", "3d1ffaff-47b5-4e63-9840-6490055fd38d", "a1b929ad-9041-4da3-bc95-d17f38820b03", "b730fa81-1758-45c0-a6c5-2ab07e4c809d", "0235942c-7bd3-46c1-8a3d-87e42a0b1e86", "033b6ad3-25fe-40e9-87fa-4f317f84478a", "e4d8ad09-a592-494d-86f3-6c2e49bbbb9c", "b3f95f3e-50ce-40cf-b5e2-4b4c8ae61982", "f3704d75-cb01-4b0b-81f2-7abeec4420da", "bad201f1-6500-4ac0-8396-30a39c0d0c79", "73520cf6-2bb8-404f-8323-0a37c20c18f9", "8e5c3116-fa82-46ee-9f4c-9760d96aea50", "e678893f-fadc-4a92-842e-e89534886b9d", "c350e8c0-8fcc-4fe0-bc47-51b024fb757a", "9699a4fb-9ff5-4a7e-8c53-2d562c9173f1", "25dfda11-00b8-432b-8c24-dcc604629be9", "619b5c69-ffb3-494d-bf19-242e453c2c05", "250dbc86-4d5c-45aa-bab5-0fb75678574b", "99525364-38e3-480a-9429-01ad97d0a2d8", "3d462e53-b907-4586-9b1f-041dde2b5dd5", "1cf6afc7-d8dd-492e-88b4-ce73f63f0525", "d9bece50-a8a4-4112-9fa6-680bd430ecdd", "c286245c-2fbd-4743-9cd9-eb9c1bbb659a", "42cfb6e3-3ebc-4a5e-bcd0-99c9fe7992b8", "a957b410-9201-41d7-bd79-92343459202d", "dfc26eb6-2c9d-4ac2-bd11-5c4dfc7e006b", "76370b7a-c03c-4d59-8dcd-d7e9c0c52f96", "2f42f669-8819-423f-9078-ec988cb71d11", "299bd409-a638-4e96-967d-1811b3f0a0ce", "15c78167-3f73-45a1-ac95-f9a426cc1da8", "35a15436-ecdd-46f4-8122-7a93d55f8130", "3e267db7-9882-49bf-a98c-6db89d6bda35", "2e9d7ff3-5614-4a6b-bd9b-6a3c2303bfd1", "8ff06795-8e4b-4e60-9a2c-f606b671e5a0", "d74309bf-30e8-4b47-9d1e-6fb074f7de7d", "bce10ef2-47dd-4eea-8c91-089e31e24736", "eec97b50-58e7-41e9-9b53-e192632655c4", "7f46f4ad-053e-4e22-be36-5df3dcdee61f", "ef4a5cf5-ade0-41f9-95e6-b351531b2b52", "017e0e6e-a191-4135-a5f0-07a2819096f5", "bc36613c-86ed-42f6-8717-2ba89c9bba7f", "9a11367a-a2a4-4b38-8bb1-8165f675e8b0", "a09c62ad-809b-43c7-bd98-d9162ea3b03e", "306b5a13-abdd-4e87-a86c-b67409f5d0ab", "4e2eb376-2f18-4e39-b42e-bd76a829f230", "84825be4-cf2f-4b36-8659-d34be70bda80", "678d2b2c-794a-46d5-b510-c7def707a498", "78d4f347-6f5a-4180-a62e-15a852049269", "2834e1b0-1323-4ee5-b819-0e836aac244e", "48d6afa8-9a0d-49b1-b080-f879108aefe9", "981e8739-a5e2-4a44-b4e2-c0d4e061ce6d", "45cba06f-441b-4ae5-8449-13a4d77805b1", "44768877-bb13-4dd9-90f0-9258cdcb92ca", "ee639ea3-ff0e-405b-b9b4-16d1c16edea8", "02a335f5-ea48-4d29-94b4-e1f50c94eb47", "e54f430b-8cdf-47d7-9c66-8688f32a256f", "72ebfaa6-fe90-48b9-a2ce-14c8713f542f", "0d8a50dc-acf2-43af-8390-ba1a0c999f71", "2e3a8f56-e8a1-4679-9ff6-1720613027d2", "bc748711-2dac-4bd6-85bb-ae2fedd0cc8b", "517cb574-ed34-4e1d-9419-bb518ce71739", "715b45d8-9afe-4831-ad94-a254697bb353", "6b8e3f25-b07a-4cbf-b3ac-e5a9f4e9d75a", "895ca622-be08-40cd-ae54-6b7de92babda", "e10a4629-75c4-45c9-9ce3-00d7b0f31232", "71841539-31c0-409d-89f1-cf8ab21edff0", "c932cbd0-306d-4159-ad30-0cd2ad018ffc", "d8276edd-e179-4a3e-818c-1f071235dbbd", "2f4abf53-ee6c-43d7-82f9-7b5d7a055ff7", "ac6ab1f8-cc31-4b40-a948-d0eb631ae68f", "aa1ff0a1-1012-420b-9db9-cdf1c988eb54", "b4be77b4-b2de-4b7a-b0ef-c13194f71569", "7eed2b0d-dbf7-48fa-b567-301dd124d1df", "e7b9eafa-47c6-48be-a365-8a8467dcff3b", "4d60d292-cbbf-4610-8ef8-cd99fe3f900f", "52b34ed8-f4fb-4fea-92dc-c5e3d19e2289", "9fd06628-d752-410e-b96c-d8a52c5eed17", "b86bdaa4-4707-4b3b-88f4-809f6f985fdb", "d24a983f-5c0b-4df2-8135-06485b4a1529", "31a66bc4-2b12-4310-919f-b8c0231e252d", "4ac75865-1dd4-4785-b8b0-ec0e133eab6c", "60151d57-1ae0-48f9-9bd9-2b55e6bf1be1", "9dffbfff-2474-4263-8580-6bffc2efe2a4", "1279b00d-cb50-4021-a595-08e7c3dd35c0", "921eb010-48e5-4aaf-9497-aff34371f106", "af97d061-29ca-4728-b3eb-3449cf8b39a2", "a105451a-ae8c-4c30-b7e5-6a678d920a6f"]
  }
}, {
  "specimens": {
    "id": "08b2a2bf-6427-11e8-bcf1-0a2705229b82",
    "identifier": [{
      "system": "PDC",
      "value": "08b2a2bf-6427-11e8-bcf1-0a2705229b82"
    }],
    "associated_project": "CPTAC-2",
    "days_to_collection": None,
    "primary_disease_type": "Colon Adenocarcinoma",
    "anatomical_site": "Not Reported",
    "source_material_type": "Primary Tumor",
    "specimen_type": "aliquot",
    "derived_from_specimen": "f6cce507-641b-11e8-bcf1-0a2705229b82",
    "derived_from_subject": "09CO022",
    "Files": ["6013ddc3-de68-4fa0-81bb-6c4b30ba06f2", "d24a983f-5c0b-4df2-8135-06485b4a1529", "033b6ad3-25fe-40e9-87fa-4f317f84478a", "7d3c33cf-5028-4ba0-98f0-c9d29411daf4", "ef4a5cf5-ade0-41f9-95e6-b351531b2b52", "f0d9fd49-298f-449b-9d29-b7253c5e2a7c", "0b236baf-d1cc-4d1b-89ab-f47a1baf2f54", "9992288d-7bdc-4cd2-99a8-5eedefbeffca", "85293896-3816-4afa-acb7-bc4311f64adc", "fbbfc3ab-0238-48e9-8dff-c669e88c54d4", "0d8a50dc-acf2-43af-8390-ba1a0c999f71", "84825be4-cf2f-4b36-8659-d34be70bda80", "25dfda11-00b8-432b-8c24-dcc604629be9", "2b85a5e2-7091-4fb3-859e-4520e38e4525", "87a94cca-986a-456c-94c4-05d297bd7274", "298465a9-a0e0-4691-b368-6a2c00be294f", "b730fa81-1758-45c0-a6c5-2ab07e4c809d", "29397011-e525-4027-9c3d-c1f86b15e041", "f07375eb-d4a0-4df9-809c-77cb576ef4b5", "d9bece50-a8a4-4112-9fa6-680bd430ecdd", "c286245c-2fbd-4743-9cd9-eb9c1bbb659a", "a957b410-9201-41d7-bd79-92343459202d", "1d03430b-fbb9-4aa4-aa63-0f1d57254bd8", "a6cbebbe-ec64-4fc4-9df9-64b7c83af3e0"]
  }
}, {
  "specimens": {
    "id": "f4af3e4d-641b-11e8-bcf1-0a2705229b82",
    "identifier": [{
      "system": "PDC",
      "value": "f4af3e4d-641b-11e8-bcf1-0a2705229b82"
    }],
    "associated_project": "CPTAC-2",
    "days_to_collection": None,
    "primary_disease_type": "Colon Adenocarcinoma",
    "anatomical_site": "Not Reported",
    "source_material_type": "Solid Tissue Normal",
    "specimen_type": "sample",
    "derived_from_specimen": "initial specimen",
    "derived_from_subject": "09CO022",
    "Files": ["0b236baf-d1cc-4d1b-89ab-f47a1baf2f54", "932ca639-c23b-4a5c-be78-5a257f9212e7", "8119ed56-ce52-43a3-a323-c4bf58dd935c", "984427aa-2013-4822-9a04-2e7e3a95e542", "a0f64b2a-3672-49a7-9a45-509d4f89a5d6", "87a94cca-986a-456c-94c4-05d297bd7274", "b576d133-59f8-40a3-8e02-8b6007547d34", "6e8310f5-0bc6-49dd-b934-06722427b421", "06b6f850-b3b6-492b-88a5-214906b71338", "8dea43ca-5b46-4d97-8fb1-ea173bc5c5d3", "85293896-3816-4afa-acb7-bc4311f64adc", "b6b5ce5c-ac41-4be9-9c02-5738509ac2f1", "3ac4db66-7660-451f-833d-32ffa4f159d6", "efcb2319-d4c1-486e-b954-5965ed777dd5", "d123999c-98fa-4171-b248-7b782643ab28", "34f3b35d-056e-458e-805d-aa9eddc8fcc9", "1f63a16d-ee0b-496f-a3cc-52eda4f199b1", "1d03430b-fbb9-4aa4-aa63-0f1d57254bd8", "f07375eb-d4a0-4df9-809c-77cb576ef4b5", "21b0644c-4bc2-4a72-826a-eb0b507add08", "e68b9b17-100e-48c1-84d7-d1a5f9485c1e", "298465a9-a0e0-4691-b368-6a2c00be294f", "c309d2e0-0551-465a-a4e0-ca5e7a59d7f1", "f0d9fd49-298f-449b-9d29-b7253c5e2a7c", "58797074-31b1-4ad2-8c4f-5bcc39486c85", "724c463d-afb5-4ab9-bf70-90505347a10f", "aeee4181-b277-472a-881d-b61bf178f597", "627d302f-557c-4e03-9050-89ee2404d04a", "72af85d8-ca3e-461d-8813-83e4981a7447", "e098e982-8a8f-449b-b36b-de20b63a8dc5", "7aece4d7-687f-428b-998b-92030e2ea0b5", "96ce4701-3f16-47dd-9778-1d8c1eef16e6", "e22e4280-7ac3-4038-888a-095b4ce4a7a6", "0d846fdf-169e-4081-910d-a792fab3df43", "40617525-424e-4bc0-8468-4e00ff9f0ccb", "2b85a5e2-7091-4fb3-859e-4520e38e4525", "5de6ec5d-e824-405d-ab47-bb633f0b30b0", "f89c439d-465d-4b2e-b117-19aa82cd87b4", "29397011-e525-4027-9c3d-c1f86b15e041", "6013ddc3-de68-4fa0-81bb-6c4b30ba06f2", "b63506ea-c6c2-41b4-9c4e-0591f3c5b620", "75b3f79d-a90e-4204-a67d-dd8af9fb2c12", "80de8307-400c-4e91-86c5-110183826a8f", "55dea7f7-c521-4b71-8eb6-4b18fde8dd3e", "d252a407-fb5b-40c4-83e4-aa4865adb9e2", "aff41011-83d2-412d-9ece-887272bbfcb1", "9992288d-7bdc-4cd2-99a8-5eedefbeffca", "22634cb8-c4a5-40a5-a9d6-e447896e8681", "f3bb04cb-8cbd-45b6-8b2c-c3c42a56e9c2", "82a24092-4d20-48be-9242-434b5cb13300", "a6cbebbe-ec64-4fc4-9df9-64b7c83af3e0", "ec2b6628-0081-4609-9d84-8fe7e8fe5ee2", "b93abf59-3591-4921-9695-3cae8898818b", "9e836fe1-d5a3-4c59-ad7a-1c8a084b6a9b", "f214f823-0d83-4563-9cfd-a4e5d421178a", "81c2839d-9f30-4f26-bca7-bc5616b1e7c7", "51906b58-0772-436f-bcb8-fc2f36fcd5e2", "7d3c33cf-5028-4ba0-98f0-c9d29411daf4", "7604aa47-6b24-4615-958b-397ecdc1e5ee", "a53aa3f0-1659-4c46-9c1c-73879ea12985", "47832160-26f2-4ad2-ae65-a781303b21d7", "fbbfc3ab-0238-48e9-8dff-c669e88c54d4", "fa9b4f1c-0297-4b48-8130-250548130a10", "1a1b004e-2ac6-42a7-b745-3e52be7fa22c", "99d06cc3-01d5-4ac1-bd89-cceee73864b5", "6f7cee54-ce35-4e89-9555-c5fe3faf0dd1", "07dde26b-a6b4-4758-8f7d-483bb4340a95", "0daea74c-6886-415b-9395-cd4e0857acb4", "41057289-39ea-4f09-b24f-dfd1336fc1ed", "8e5af17a-02fc-4ba7-9887-bc03b0c83fca", "cfb64703-2cdb-4b35-984f-4d0a582aff06", "2cec8163-ac36-45ba-bdf6-ece8372dd44a", "ebd17416-a9ee-4ce9-9252-76611337a18e", "3f3383b2-97a7-4db3-9055-ef1190adca92", "0a63771a-14b2-4d6b-9c08-c7177c372789", "df3f15b4-15ce-4957-8c30-912eedf71e18", "6ca7d2db-8d2e-4b3a-b296-18b401a8a447", "6e2d5918-e230-433d-9178-7c8fb348d19b", "7f475e53-7efa-45f4-8e34-3bc84cc068b8", "7b831b12-9e11-489f-bd37-47d37d740d4d", "28cebeba-0af6-42f8-9d3d-9b2d6aadcd8a", "cea9dd09-9672-4bf5-b093-0227db5b13f0", "3d1ffaff-47b5-4e63-9840-6490055fd38d", "a1b929ad-9041-4da3-bc95-d17f38820b03", "b730fa81-1758-45c0-a6c5-2ab07e4c809d", "0235942c-7bd3-46c1-8a3d-87e42a0b1e86", "033b6ad3-25fe-40e9-87fa-4f317f84478a", "e4d8ad09-a592-494d-86f3-6c2e49bbbb9c", "b3f95f3e-50ce-40cf-b5e2-4b4c8ae61982", "f3704d75-cb01-4b0b-81f2-7abeec4420da", "bad201f1-6500-4ac0-8396-30a39c0d0c79", "73520cf6-2bb8-404f-8323-0a37c20c18f9", "8e5c3116-fa82-46ee-9f4c-9760d96aea50", "e678893f-fadc-4a92-842e-e89534886b9d", "c350e8c0-8fcc-4fe0-bc47-51b024fb757a", "9699a4fb-9ff5-4a7e-8c53-2d562c9173f1", "25dfda11-00b8-432b-8c24-dcc604629be9", "619b5c69-ffb3-494d-bf19-242e453c2c05", "250dbc86-4d5c-45aa-bab5-0fb75678574b", "99525364-38e3-480a-9429-01ad97d0a2d8", "3d462e53-b907-4586-9b1f-041dde2b5dd5", "1cf6afc7-d8dd-492e-88b4-ce73f63f0525", "d9bece50-a8a4-4112-9fa6-680bd430ecdd", "c286245c-2fbd-4743-9cd9-eb9c1bbb659a", "42cfb6e3-3ebc-4a5e-bcd0-99c9fe7992b8", "a957b410-9201-41d7-bd79-92343459202d", "dfc26eb6-2c9d-4ac2-bd11-5c4dfc7e006b", "76370b7a-c03c-4d59-8dcd-d7e9c0c52f96", "2f42f669-8819-423f-9078-ec988cb71d11", "299bd409-a638-4e96-967d-1811b3f0a0ce", "15c78167-3f73-45a1-ac95-f9a426cc1da8", "35a15436-ecdd-46f4-8122-7a93d55f8130", "3e267db7-9882-49bf-a98c-6db89d6bda35", "2e9d7ff3-5614-4a6b-bd9b-6a3c2303bfd1", "8ff06795-8e4b-4e60-9a2c-f606b671e5a0", "d74309bf-30e8-4b47-9d1e-6fb074f7de7d", "bce10ef2-47dd-4eea-8c91-089e31e24736", "eec97b50-58e7-41e9-9b53-e192632655c4", "7f46f4ad-053e-4e22-be36-5df3dcdee61f", "ef4a5cf5-ade0-41f9-95e6-b351531b2b52", "017e0e6e-a191-4135-a5f0-07a2819096f5", "bc36613c-86ed-42f6-8717-2ba89c9bba7f", "9a11367a-a2a4-4b38-8bb1-8165f675e8b0", "a09c62ad-809b-43c7-bd98-d9162ea3b03e", "306b5a13-abdd-4e87-a86c-b67409f5d0ab", "4e2eb376-2f18-4e39-b42e-bd76a829f230", "84825be4-cf2f-4b36-8659-d34be70bda80", "678d2b2c-794a-46d5-b510-c7def707a498", "78d4f347-6f5a-4180-a62e-15a852049269", "2834e1b0-1323-4ee5-b819-0e836aac244e", "48d6afa8-9a0d-49b1-b080-f879108aefe9", "981e8739-a5e2-4a44-b4e2-c0d4e061ce6d", "45cba06f-441b-4ae5-8449-13a4d77805b1", "44768877-bb13-4dd9-90f0-9258cdcb92ca", "ee639ea3-ff0e-405b-b9b4-16d1c16edea8", "02a335f5-ea48-4d29-94b4-e1f50c94eb47", "e54f430b-8cdf-47d7-9c66-8688f32a256f", "72ebfaa6-fe90-48b9-a2ce-14c8713f542f", "0d8a50dc-acf2-43af-8390-ba1a0c999f71", "2e3a8f56-e8a1-4679-9ff6-1720613027d2", "bc748711-2dac-4bd6-85bb-ae2fedd0cc8b", "517cb574-ed34-4e1d-9419-bb518ce71739", "715b45d8-9afe-4831-ad94-a254697bb353", "6b8e3f25-b07a-4cbf-b3ac-e5a9f4e9d75a", "895ca622-be08-40cd-ae54-6b7de92babda", "e10a4629-75c4-45c9-9ce3-00d7b0f31232", "71841539-31c0-409d-89f1-cf8ab21edff0", "c932cbd0-306d-4159-ad30-0cd2ad018ffc", "d8276edd-e179-4a3e-818c-1f071235dbbd", "2f4abf53-ee6c-43d7-82f9-7b5d7a055ff7", "ac6ab1f8-cc31-4b40-a948-d0eb631ae68f", "aa1ff0a1-1012-420b-9db9-cdf1c988eb54", "b4be77b4-b2de-4b7a-b0ef-c13194f71569", "7eed2b0d-dbf7-48fa-b567-301dd124d1df", "e7b9eafa-47c6-48be-a365-8a8467dcff3b", "4d60d292-cbbf-4610-8ef8-cd99fe3f900f", "52b34ed8-f4fb-4fea-92dc-c5e3d19e2289", "9fd06628-d752-410e-b96c-d8a52c5eed17", "b86bdaa4-4707-4b3b-88f4-809f6f985fdb", "d24a983f-5c0b-4df2-8135-06485b4a1529", "31a66bc4-2b12-4310-919f-b8c0231e252d", "4ac75865-1dd4-4785-b8b0-ec0e133eab6c", "60151d57-1ae0-48f9-9bd9-2b55e6bf1be1", "9dffbfff-2474-4263-8580-6bffc2efe2a4", "1279b00d-cb50-4021-a595-08e7c3dd35c0", "921eb010-48e5-4aaf-9497-aff34371f106", "af97d061-29ca-4728-b3eb-3449cf8b39a2", "a105451a-ae8c-4c30-b7e5-6a678d920a6f"]
  }
}, {
  "specimens": {
    "id": "208ebc64-6425-11e8-bcf1-0a2705229b82",
    "identifier": [{
      "system": "PDC",
      "value": "208ebc64-6425-11e8-bcf1-0a2705229b82"
    }],
    "associated_project": "CPTAC-2",
    "days_to_collection": None,
    "primary_disease_type": "Colon Adenocarcinoma",
    "anatomical_site": "Not Reported",
    "source_material_type": "Solid Tissue Normal",
    "specimen_type": "aliquot",
    "derived_from_specimen": "f4af3e4d-641b-11e8-bcf1-0a2705229b82",
    "derived_from_subject": "09CO022",
    "Files": ["3d462e53-b907-4586-9b1f-041dde2b5dd5", "75b3f79d-a90e-4204-a67d-dd8af9fb2c12", "80de8307-400c-4e91-86c5-110183826a8f", "48d6afa8-9a0d-49b1-b080-f879108aefe9", "55dea7f7-c521-4b71-8eb6-4b18fde8dd3e", "8119ed56-ce52-43a3-a323-c4bf58dd935c", "1cf6afc7-d8dd-492e-88b4-ce73f63f0525", "984427aa-2013-4822-9a04-2e7e3a95e542", "aff41011-83d2-412d-9ece-887272bbfcb1", "a0f64b2a-3672-49a7-9a45-509d4f89a5d6", "ee639ea3-ff0e-405b-b9b4-16d1c16edea8", "42cfb6e3-3ebc-4a5e-bcd0-99c9fe7992b8", "81c2839d-9f30-4f26-bca7-bc5616b1e7c7", "f214f823-0d83-4563-9cfd-a4e5d421178a", "b6b5ce5c-ac41-4be9-9c02-5738509ac2f1", "51906b58-0772-436f-bcb8-fc2f36fcd5e2", "2834e1b0-1323-4ee5-b819-0e836aac244e", "a53aa3f0-1659-4c46-9c1c-73879ea12985", "3ac4db66-7660-451f-833d-32ffa4f159d6", "efcb2319-d4c1-486e-b954-5965ed777dd5", "d123999c-98fa-4171-b248-7b782643ab28", "47832160-26f2-4ad2-ae65-a781303b21d7", "bc748711-2dac-4bd6-85bb-ae2fedd0cc8b", "517cb574-ed34-4e1d-9419-bb518ce71739", "fa9b4f1c-0297-4b48-8130-250548130a10", "715b45d8-9afe-4831-ad94-a254697bb353", "895ca622-be08-40cd-ae54-6b7de92babda", "2f42f669-8819-423f-9078-ec988cb71d11", "dfc26eb6-2c9d-4ac2-bd11-5c4dfc7e006b", "1a1b004e-2ac6-42a7-b745-3e52be7fa22c", "21b0644c-4bc2-4a72-826a-eb0b507add08", "0daea74c-6886-415b-9395-cd4e0857acb4", "41057289-39ea-4f09-b24f-dfd1336fc1ed", "8e5af17a-02fc-4ba7-9887-bc03b0c83fca", "ac6ab1f8-cc31-4b40-a948-d0eb631ae68f", "cfb64703-2cdb-4b35-984f-4d0a582aff06", "ebd17416-a9ee-4ce9-9252-76611337a18e", "58797074-31b1-4ad2-8c4f-5bcc39486c85", "aeee4181-b277-472a-881d-b61bf178f597", "0a63771a-14b2-4d6b-9c08-c7177c372789", "d74309bf-30e8-4b47-9d1e-6fb074f7de7d", "627d302f-557c-4e03-9050-89ee2404d04a", "bce10ef2-47dd-4eea-8c91-089e31e24736", "6ca7d2db-8d2e-4b3a-b296-18b401a8a447", "6e2d5918-e230-433d-9178-7c8fb348d19b", "72af85d8-ca3e-461d-8813-83e4981a7447", "e7b9eafa-47c6-48be-a365-8a8467dcff3b", "4d60d292-cbbf-4610-8ef8-cd99fe3f900f", "7f475e53-7efa-45f4-8e34-3bc84cc068b8", "52b34ed8-f4fb-4fea-92dc-c5e3d19e2289", "7b831b12-9e11-489f-bd37-47d37d740d4d", "e22e4280-7ac3-4038-888a-095b4ce4a7a6", "7f46f4ad-053e-4e22-be36-5df3dcdee61f", "28cebeba-0af6-42f8-9d3d-9b2d6aadcd8a", "3d1ffaff-47b5-4e63-9840-6490055fd38d", "0d846fdf-169e-4081-910d-a792fab3df43", "31a66bc4-2b12-4310-919f-b8c0231e252d", "a1b929ad-9041-4da3-bc95-d17f38820b03", "40617525-424e-4bc0-8468-4e00ff9f0ccb", "0235942c-7bd3-46c1-8a3d-87e42a0b1e86", "4ac75865-1dd4-4785-b8b0-ec0e133eab6c", "f89c439d-465d-4b2e-b117-19aa82cd87b4", "e4d8ad09-a592-494d-86f3-6c2e49bbbb9c", "678d2b2c-794a-46d5-b510-c7def707a498", "f3704d75-cb01-4b0b-81f2-7abeec4420da", "1279b00d-cb50-4021-a595-08e7c3dd35c0", "bad201f1-6500-4ac0-8396-30a39c0d0c79", "78d4f347-6f5a-4180-a62e-15a852049269", "a105451a-ae8c-4c30-b7e5-6a678d920a6f", "b63506ea-c6c2-41b4-9c4e-0591f3c5b620", "619b5c69-ffb3-494d-bf19-242e453c2c05", "250dbc86-4d5c-45aa-bab5-0fb75678574b"]
  }
}, {
  "specimens": {
    "id": "44f5b956-642a-11e8-bcf1-0a2705229b82",
    "identifier": [{
      "system": "PDC",
      "value": "44f5b956-642a-11e8-bcf1-0a2705229b82"
    }],
    "associated_project": "CPTAC-2",
    "days_to_collection": None,
    "primary_disease_type": "Colon Adenocarcinoma",
    "anatomical_site": "Not Reported",
    "source_material_type": "Primary Tumor",
    "specimen_type": "aliquot",
    "derived_from_specimen": "f6cce507-641b-11e8-bcf1-0a2705229b82",
    "derived_from_subject": "09CO022",
    "Files": ["99525364-38e3-480a-9429-01ad97d0a2d8", "af97d061-29ca-4728-b3eb-3449cf8b39a2", "932ca639-c23b-4a5c-be78-5a257f9212e7", "d252a407-fb5b-40c4-83e4-aa4865adb9e2", "45cba06f-441b-4ae5-8449-13a4d77805b1", "22634cb8-c4a5-40a5-a9d6-e447896e8681", "f3bb04cb-8cbd-45b6-8b2c-c3c42a56e9c2", "82a24092-4d20-48be-9242-434b5cb13300", "02a335f5-ea48-4d29-94b4-e1f50c94eb47", "b576d133-59f8-40a3-8e02-8b6007547d34", "e54f430b-8cdf-47d7-9c66-8688f32a256f", "6e8310f5-0bc6-49dd-b934-06722427b421", "06b6f850-b3b6-492b-88a5-214906b71338", "ec2b6628-0081-4609-9d84-8fe7e8fe5ee2", "9e836fe1-d5a3-4c59-ad7a-1c8a084b6a9b", "b93abf59-3591-4921-9695-3cae8898818b", "72ebfaa6-fe90-48b9-a2ce-14c8713f542f", "8dea43ca-5b46-4d97-8fb1-ea173bc5c5d3", "7604aa47-6b24-4615-958b-397ecdc1e5ee", "34f3b35d-056e-458e-805d-aa9eddc8fcc9", "2e3a8f56-e8a1-4679-9ff6-1720613027d2", "76370b7a-c03c-4d59-8dcd-d7e9c0c52f96", "1f63a16d-ee0b-496f-a3cc-52eda4f199b1", "6b8e3f25-b07a-4cbf-b3ac-e5a9f4e9d75a", "e10a4629-75c4-45c9-9ce3-00d7b0f31232", "299bd409-a638-4e96-967d-1811b3f0a0ce", "99d06cc3-01d5-4ac1-bd89-cceee73864b5", "15c78167-3f73-45a1-ac95-f9a426cc1da8", "35a15436-ecdd-46f4-8122-7a93d55f8130", "71841539-31c0-409d-89f1-cf8ab21edff0", "6f7cee54-ce35-4e89-9555-c5fe3faf0dd1", "3e267db7-9882-49bf-a98c-6db89d6bda35", "07dde26b-a6b4-4758-8f7d-483bb4340a95", "c932cbd0-306d-4159-ad30-0cd2ad018ffc", "e68b9b17-100e-48c1-84d7-d1a5f9485c1e", "d8276edd-e179-4a3e-818c-1f071235dbbd", "2f4abf53-ee6c-43d7-82f9-7b5d7a055ff7", "c309d2e0-0551-465a-a4e0-ca5e7a59d7f1", "aa1ff0a1-1012-420b-9db9-cdf1c988eb54", "2e9d7ff3-5614-4a6b-bd9b-6a3c2303bfd1", "724c463d-afb5-4ab9-bf70-90505347a10f", "3f3383b2-97a7-4db3-9055-ef1190adca92", "df3f15b4-15ce-4957-8c30-912eedf71e18", "b4be77b4-b2de-4b7a-b0ef-c13194f71569", "8ff06795-8e4b-4e60-9a2c-f606b671e5a0", "7eed2b0d-dbf7-48fa-b567-301dd124d1df", "2cec8163-ac36-45ba-bdf6-ece8372dd44a", "e098e982-8a8f-449b-b36b-de20b63a8dc5", "7aece4d7-687f-428b-998b-92030e2ea0b5", "96ce4701-3f16-47dd-9778-1d8c1eef16e6", "9fd06628-d752-410e-b96c-d8a52c5eed17", "eec97b50-58e7-41e9-9b53-e192632655c4", "b86bdaa4-4707-4b3b-88f4-809f6f985fdb", "bc36613c-86ed-42f6-8717-2ba89c9bba7f", "017e0e6e-a191-4135-a5f0-07a2819096f5", "cea9dd09-9672-4bf5-b093-0227db5b13f0", "9a11367a-a2a4-4b38-8bb1-8165f675e8b0", "a09c62ad-809b-43c7-bd98-d9162ea3b03e", "306b5a13-abdd-4e87-a86c-b67409f5d0ab", "5de6ec5d-e824-405d-ab47-bb633f0b30b0", "4e2eb376-2f18-4e39-b42e-bd76a829f230", "b3f95f3e-50ce-40cf-b5e2-4b4c8ae61982", "60151d57-1ae0-48f9-9bd9-2b55e6bf1be1", "9dffbfff-2474-4263-8580-6bffc2efe2a4", "73520cf6-2bb8-404f-8323-0a37c20c18f9", "8e5c3116-fa82-46ee-9f4c-9760d96aea50", "e678893f-fadc-4a92-842e-e89534886b9d", "921eb010-48e5-4aaf-9497-aff34371f106", "c350e8c0-8fcc-4fe0-bc47-51b024fb757a", "9699a4fb-9ff5-4a7e-8c53-2d562c9173f1", "981e8739-a5e2-4a44-b4e2-c0d4e061ce6d", "44768877-bb13-4dd9-90f0-9258cdcb92ca"]
  }
}]
new_dat = {}
for spec in fore_ex_specs:
    new_dat[spec['specimens']['id']]= spec['specimens']

In [ ]:
tree = {}
for idd, spec in new_dat.items():
    if spec['specimen_type']=='sample':
        tree[spec['id']] = [] 
    else:
        if spec["derived_from_specimen"] in tree:
            tree[spec["derived_from_specimen"]].append(spec['id'])
tree

In [ ]:
for sample, aliquots in tree.items():
    samp_files = set(new_dat[sample]['Files'])
    aliquot_files = []
    for aliquot in aliquots:
        aliquot_files.extend(new_dat[aliquot]['Files'])
    aliquot_files = set(aliquot_files)
    if samp_files != aliquot_files:
        print(samp_files-aliquot_files)

In [ ]:
samp_1_files = set(new_dat['f6cce507-641b-11e8-bcf1-0a2705229b82']['Files'])
samp_2_files = set(new_dat['f4af3e4d-641b-11e8-bcf1-0a2705229b82']['Files'])
samp_1_files == samp_2_files

In [ ]:
import requests
from string import Template
query = """{ fileMetadata(file_id: "c286245c-2fbd-4743-9cd9-eb9c1bbb659a" acceptDUA: true) {
file_id
file_name
file_location
md5sum
file_size
file_submitter_id
data_category
file_type
file_format
experiment_type
aliquots {
  sample_id
  aliquot_id
  case_id
}
}
}"""

url = 'https://pdc.cancer.gov/graphql'
response = requests.post(url, json={'query': query})

if(response.ok):
  #If the response was OK then print the returned JSON
  jData = json.loads(response.content)

  print (json.dumps(jData, indent=4, sort_keys=True))
else:
  # If response code is not ok (200), print the resulting http error code with description
  response.raise_for_status()

In [ ]:
with gzip.open('../data/pdc.files-specimens-cases.json.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for f in reader:
        if f['file_id'] == "c286245c-2fbd-4743-9cd9-eb9c1bbb659a":
            print(f)
            break
            

In [ ]:
with gzip.open('../pdc.all_files.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for f in reader:
        if f['file_id'] == "c286245c-2fbd-4743-9cd9-eb9c1bbb659a":
            file_rec = f
            break

In [ ]:
print(len(file_rec['cases']))

In [ ]:
for case in file_rec['cases']:
    print(case['case_id'])
    

In [ ]:
cases = []
with gzip.open('../pdc.all_cases.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for f in reader:
        if f['case_id'] == "459e3b69-63d6-11e8-bcf1-0a2705229b82":
            cases.append(f)


In [ ]:
samp_files = []
for case in cases:
    print(case['study'])
    for sample in case['samples']:
        if sample['sample_id'] != 'f6cce507-641b-11e8-bcf1-0a2705229b82':
            for file in sample['files']:
                samp_files.append(file['file_id'])
            print(list(sample.keys()))
    #print(case['samples'])
samp_files = set(samp_files)
"c286245c-2fbd-4743-9cd9-eb9c1bbb659a" in samp_files

In [ ]:
with open('../files_per_sample_dict.json') as pdf:
    files_sample_dict = json.load(pdf)
files_sample_dict['f4af3e4d-641b-11e8-bcf1-0a2705229b82']

In [ ]:
#samp_files = []
with gzip.open('../pdc.all_cases.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for case in reader:
        for sample in case['samples']:
            for aliquot in sample['aliquots']:
                if aliquot['aliquot_id'] == '4c2b9219-6423-11e8-bcf1-0a2705229b82':
                    print(aliquot)
            
            #for file in sample['files']:
            #    samp_files.append(file['file_id'])
            #print(sample['sample_id'])
    #print(case['samples'])
#samp_files = set(samp_files)
#"c286245c-2fbd-4743-9cd9-eb9c1bbb659a" in samp_files

In [ ]:
files_sample_dict['4c2b9219-6423-11e8-bcf1-0a2705229b82']

In [ ]:
study_files = []
with gzip.open('../pdc.all_cases.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for case in reader:
        if case['study']['pdc_study_id']=='PDC000117':
            print('found')
            for sample in case['samples']:
                study_files.extend(sample['files'])
            #print(case['samples'])
study_files

In [ ]:
study_f_list = []
for f in study_files:
    study_f_list.append(f['file_id'])
study_f_list = set(study_f_list)

In [ ]:
study_f_list

In [ ]:
len(study_f_list)

In [ ]:
count = 0
with gzip.open('../pdc.all_cases.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for case in reader:
        count+=1
count

In [ ]:
count = 0
with gzip.open('../pdc.all_cases.H.jsonl.gz') as pdf:
    reader = jsonlines.Reader(pdf)
    for case in reader:
        count+=1
count